In [53]:
#import packages
import numpy as np
from sklearn import linear_model

In [95]:
#Defining some useful functions

def Rademacher_matrix(d,n):
    """
    This fucntion generates a Rademacher matrix
    """
    return np.random.choice([-1, 1], size=(d,n))



def Rademacher_matrix_concatenated(d,n):
    """
    This function generates a Rademacher matrix and add a line of ones
    """
    Z=Rademacher_matrix(d,n)
    Last_line_of_ones = np.ones((1, Z.shape[1]))
    return (Z,np.concatenate((Z, Last_line_of_ones), axis=0))



def Lasso_reg(Y_tilde,Z):
    """
    This function gives the solution to the Lasso regression in a multivariate model
    """
    lasso = linear_model.LassoCV(cv=5)
    lasso.fit(Z,Y_tilde )
    g = lasso.coef_
    u=lasso.intercept_
    return(g,u)


def GradiantEstimate(x_t_vect:np.ndarray,d,n,delta,f):
    """
    This function corresponds to the pseudo algorithme 1 defined in the paper
    """
    Z=Rademacher_matrix(d,n)
    y_t_vect=np.zeros(n)
    for i in range(n):
        z_i=Z[:, i]
        y_t_vect[i] = f(x_t_vect+delta*z_i) #Construct the vector y_t:= f(x_t+delta*z_i)+noise. We suppose that f returns the true output plus noise
    y_tilde=y_t_vect/delta
    (g,u)=Lasso_reg(Y_tilde=y_tilde,Z=Z.T)
    return(g,u)


def random_vector_unit_spehre(d):
    random_vector = np.random.normal(size=d)
    return(random_vector/np.linalg.norm(random_vector))



def BGD(delta,T_prime,f,d,y_t,S):
        for t_prime in range (T_prime):
             x_t=y_t+delta*random_vector_unit_spehre(d) #update x_t at each step where x_t=y_t+delta*u_t.
             restriction_to_S=np.where(np.isin(x_t, S), x_t, 0) # Select the elements of x_t that are in S and set the coefficients of the remaining elements to 0.
             f_ST=f(restriction_to_S) # Compute the fonction f where the input is the vector restricted to S
             y_t=y_t-f_ST*(len(S)/delta)
        return x_t




def Successive_selection_algo(T,eta,delta,f,s,B,d,n):
    x_t_vect=np.zeros(d)
    S_hat_t=[]
    S_hat_t_minus_1=[1]
    #chi_tilde={x for x in chi if np.linalg.norm(x,ord=1)<=B}
    T_prime=int(T/2)
    t=0
    while len(S_hat_t)<s and t<s and len(S_hat_t)!=len(S_hat_t_minus_1): # setting  conditions to stop the while loop 

        S_hat_t_copy=S_hat_t.copy() # creating a copy of S_hat_t that will be used to update S_hat_t_minus_1
        t=t+1
        g_hat_t,u_t=GradiantEstimate(x_t_vect,d,n,delta,f) # Estimate the gradient g_t 
        right_set=[i for i in range(d) if abs(g_hat_t[i]) >= eta]
        S_hat_t=S_hat_t_minus_1 + (right_set) # Thresholding
        S_hat_t_minus_1=S_hat_t_copy # update S_hat_minus_1
        x_t_minus_one=x_t_vect.copy() # Keep a copy of x_t to use it for the output 
        x_t_vect=BGD(T_prime=T_prime,f=f,d=d,y_t=x_t_vect,S=S_hat_t,delta=delta) # performe the finite difference algorithme that returns x_t
    if len(S_hat_t)==s:
         return(x_t_vect)
    else:
         return(x_t_minus_one)





In [103]:
# Functions that will be used for testing 
def f_test(x_t,noise=1):#S le support 
    return(np.linalg.norm(x_t)+noise*np.random.normal(0,1,1))
def vect_f_test(x_t, delta, d, n):
    y_t_vecteur = np.zeros(n)
    for i in range(n):
        y_t_vecteur[i] = f_test(x_t, delta, d)
    y_t = f_test(x_t=x_t, delta=0, d=d, noise=0)
    return y_t_vecteur, y_t
def f_test_S(X_t,S,noise=1):
    X_S=np.array([X_t[i] if i in S else 0 for i in range(len(X_t))])
    return(np.linalg.norm(X_S)+np.sum(X_S))+noise*np.random.normal(0,1,1)



In [56]:
#precising the parameters 
d=100
n=50
delta=0.5
lamda=0.1
x_t=np.random.binomial(1, 1/2,size=(d,))

In [94]:
g_t,u_t=GradiantEstimate(x_t,d,n,delta,f_test)

In [73]:
g_t.shape

(100,)

In [52]:
print(f" The lasso estimator of g_t and u_t are:\n  g_t= {g_t} \n u_t= {u_t} ")

 The lasso estimator of g_t and u_t are:
  g_t=[ 0.85584678 -0.         -0.27518014  0.         -0.92315559  0.
 -0.13270757 -0.49917827 -0.         -0.          1.27936878 -0.
  0.          0.          0.         -0.          0.          0.04199
 -0.49251199 -0.          0.1991393  -0.42375978  0.         -0.
 -0.         -0.          0.          1.14341817  0.         -1.11363068
  0.         -0.48977127 -0.         -0.46033951 -0.          0.
 -0.23029969  0.60316216  0.34387393  0.30580728 -0.20423971  0.
  0.          0.14773199 -0.         -0.         -0.          0.
 -0.07978006 -0.30870114  0.         -0.09659295 -2.60461444  0.32942071
  0.          0.          0.         -0.          1.25801597 -0.
 -0.         -0.97083662  1.30076493 -0.64672034  0.         -0.
 -0.43377556 -0.          0.         -0.          0.         -0.57633336
 -0.3184277  -0.04609787 -0.88207381  0.         -0.          0.
  0.          0.          0.09776046  0.          0.         -1.0050212
  0.068

In [96]:
Successive_selection_algo(T=100,eta=10,delta=2,f=f_test,s=10,B=2,d=d,n=n)

/Users/ilyeshammouda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.138e-02, tolerance: 5.737e-03
  model = cd_fast.enet_coordinate_descent(
/Users/ilyeshammouda/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.681e-02, tolerance: 7.286e-03
  model = cd_fast.enet_coordinate_descent(


In [104]:
my_vector = [1, 2, 3, 4, 5]
indices = {1, 3}
f_test_S(my_vector,indices)

array([11.02488653])